<a href="https://colab.research.google.com/github/neuhausmatheus/vicuna/blob/main/1508_Vicuna_13B_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may encounter an error when installing flash-attn. I couldn't figure it out. Maybe you can.**

Solution -1

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
%%cuda --name testGoogleColab.cu

#include <stdio.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char* file, int line, bool abort = true)
{
    if (code != cudaSuccess)
    {
        fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
        if (abort) exit(code);
    }
}

__global__ void add(int a, int b, int *c) {
   *c = a + b; }

int main() {

   // --- Host declarations and initializations
   int a, b, c;
   a = 2;
   b = 6;

   // --- Device allocations
   int *d_c; gpuErrchk(cudaMalloc(&d_c, sizeof(int)));

   // --- Kernel execution
   add<<<1,1>>>(a, b, d_c);
   gpuErrchk(cudaPeekAtLastError());
   gpuErrchk(cudaDeviceSynchronize());

   // --- Moving the results from device to host
   gpuErrchk(cudaMemcpy(&c, d_c, sizeof(int), cudaMemcpyDeviceToHost));

   // --- Results printout
   printf("%d + %d is %d\n", a, b, c);
   
   return 0; }




Solution 0

In [ ]:
! add-apt-repository ppa:graphics-drivers/ppa
! apt update
! apt install nvidia-384 nvidia-384-dev
! apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev
! nvidia-smi
!gcc -v
!apt install gcc-6
!apt install g++-6
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/patches/1/cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
!dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
!dpkg -i cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
!ls


Solution 1

In [ ]:
!lsb_release -a
!nvcc --version
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.7.0/local_installers/cuda-!repo-ubuntu1804-11-7-local_11.7.0-515.43.04-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-7-local_11.7.0-515.43.04-1_amd64.deb
!cp /var/cuda-repo-ubuntu1804-11-7-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-11-7
!nvcc --version

Solution 2 

In [ ]:
!cat /usr/local/lib/python3.6/dist-packages/external/local_config_cuda/cuda/cuda/cuda_config.h |\
grep TF_CUDA_VERSION

In [ ]:
!pip install mxnet-cu80


Solution 3

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
!pip install mxnet-cu92
!nvcc --version

Testing GPU access on Colab

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

!nvcc --version
!echo "name of the GPUS:"
!echo $gpu_id

Fri Apr  7 17:07:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Testing Memory access on Colab

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


Preparation...

In [7]:
%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!cd ~
!git clone https://github.com/huggingface/transformers.git && cd transformers && git checkout cae78c46 && pip install .
# Install fastchat
!pip3 install --upgrade pip
!git clone https://github.com/lm-sys/FastChat && cd FastChat && pip install -e .
%pip install einops
!mkdir checkpoints
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl
%pip install flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the 

In [8]:
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
!mkdir models
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V2_unfiltered_cleaned_split.json
!python download-model.py decapoda-research/llama-13b-hf

--2023-04-07 17:12:03--  https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9424 (9.2K) [text/plain]
Saving to: ‘download-model.py’

download-model.py   100%[===================>]   9.20K  --.-KB/s    in 0s      

2023-04-07 17:12:04 (98.3 MB/s) - ‘download-model.py’ saved [9424/9424]

--2023-04-07 17:12:04--  https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V2_unfiltered_cleaned_split.json
Resolving huggingface.co (huggingface.co)... 13.224.249.63, 13.224.249.89, 13.224.249.35, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.63|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://

**Manually edit tokenizer_config.json to: {"bos_token": "", "eos_token": "", "model_max_length": 2048, "tokenizer_class": "LlamaTokenizer", "unk_token": ""}**

**Enter wandb api key**

In [ ]:
%pip install wandb
import wandb
wandb.login()

**8 x A100 80gb training run**  (CUDA_VISIBLE_DEVICES=0 && NVIDIA_VISIBLE_DEVICES=$gpu_id)

Rename: ShareGPT_unfiltered_cleaned_split.json or     --data_path ShareGPT_unfiltered_cleaned_split.json \


Reduce the batch size: One of the easiest ways to reduce GPU memory usage is by reducing the batch size. In the code you provided, you can try reducing the per_device_train_batch_size and per_device_eval_batch_size arguments to a smaller value. However, this may increase the training time since it takes more iterations to cover the entire dataset.

Use mixed precision training: Another way to reduce GPU memory usage is by using mixed precision training. This involves using lower precision floating-point numbers (e.g., float16 instead of float32) for some parts of the computation, which reduces memory usage and can speed up training. In the code you provided, you can try setting the bf16 and tf32 arguments to False to disable mixed precision training.

Use gradient accumulation: If reducing the batch size is not an option, you can try using gradient accumulation. This involves accumulating the gradients over multiple iterations before updating the model parameters, which reduces the memory usage per iteration. In the code you provided, you can try increasing the gradient_accumulation_steps argument to a larger value (e.g., 2, 4, etc.).

Use smaller models: If none of the above options work, you can try using a smaller model. In the code you provided, you can try using a smaller pre-trained model by setting the model_name_or_path argument to a smaller model (e.g., "distilgpt2" instead of "decapoda-research_llama-13b-hf"). However, this may reduce the model's performance on the task

In [20]:
!export CUDA_LAUNCH_BLOCKING=1 && echo $CUDA_LAUNCH_BLOCKING && unset CUDA_VISIBLE_DEVICES && export CUDA_VISIBLE_DEVICES=0 && echo $CUDA_VISIBLE_DEVICES && \
torchrun --nnodes=1 --nproc_per_node=1 --master_port=21001 \
    FastChat/fastchat/train/train.py \
    --model_name_or_path models/decapoda-research_llama-13b-hf \
    --data_path ShareGPT_unfiltered_cleaned_split.json \
    --bf16 False \
    --output_dir ./checkpoints \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 100 \
    --learning_rate 4e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 False \
    --model_max_length 1024 \
    --gradient_checkpointing False \
    --lazy_preprocess True

1
0
2023-04-07 18:17:26.347634: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/transformers/training_args.py:1356: FutureWarning: using `--fsdp_transformer_layer_cls_to_wrap` is deprecated. Use fsdp_config instead 
  warnings.warn(
Loading checkpoint shards: 100% 41/41 [02:35<00:00,  3.79s/it]
Using pad_token, but it is not set yet.
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /content/FastChat/fastchat/train/train.py:340 in <module>                    │
│                                                                              │
│   337                                                                        │
│   338                                                                        │
│   339 if __name__ == "__main__":                                             │
│ ❱ 340 │   train()                                                            │
│   341   